# Stochastic Variational Method with RL algorithms

In [1]:
import numpy as np
import gym
import torch
import subprocess
import os
import torch
import h5py

## Expoloring environment

In [2]:
env = gym.make('svm_env:svmEnv-v1', file_sigmas ="./svmCodeSVD/sigmas.dat" )

print('### Env Name ######', env.unwrapped.spec.id)

obs_space = env.observation_space

print('###### Observation space ####### \n', obs_space)

state_size = env.observation_space.shape[-1]

print('###### Size of observation space ####### \n', state_size)

act_space = env.action_space

print('###### Action space ####### \n', act_space)

act_size = env.action_space.shape[-1]

print('###### Number of actions ####### \n', act_size)

state = env.reset()

print('##### State after reset ###### \n', state)

print('##### File where will be stored sigmas \n', env.file_sigmas)



### Env Name ###### svmEnv-v1
###### Observation space ####### 
 Box(-inf, inf, (1,), float32)
###### Size of observation space ####### 
 1
###### Action space ####### 
 Box(-1.0, 1.0, (3,), float32)
###### Number of actions ####### 
 3
#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
##### State after reset ###### 
 [0.]
##### File where will be stored sigmas 
 ./svmCodeSVD/sigmas.dat


## Saving folders

In [ ]:
models_dir_ppo = 'models/PPO'
models_dir_td3 = 'models/TD3'

logdir = 'logs'

if not os.path.exists(models_dir_ppo):
    os.makedirs(models_dir_ppo)
    
if not os.path.exists(models_dir_td3):
    os.makedirs(models_dir_td3)

if not os.path.exists(logdir):
    os.makedirs(logdir)

## Twin Delayed DDPG (TD3) from `stable_baseline3`

In [ ]:
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

# The noise objects for DDPG
action_noise = NormalActionNoise(mean=np.zeros(act_size), sigma=0.2 * np.ones(act_size))

model = DDPG("MlpPolicy", env, action_noise=action_noise, batch_size=64, gamma=1.0, verbose=1, seed=0
            , tensorboard_log=logdir )

# (policy, env, learning_rate=0.001, buffer_size=1000000,learning_starts=100, batch_size=100, 
# tau=0.005, gamma=0.99, train_freq=(1, 'episode'),  gradient_steps=- 1, action_noise=None, 
# replay_buffer_class=None, replay_buffer_kwargs=None,  optimize_memory_usage=False, 
# tensorboard_log=None, create_eval_env=False, policy_kwargs=None,  verbose=0, seed=None, 
# device='auto', _init_setup_model=True)

model.learn(total_timesteps=300, log_interval = 5, n_eval_episodes = 1)

# learn(total_timesteps, callback=None, log_interval=4, eval_env=None, eval_freq=- 1,
# n_eval_episodes=5, tb_log_name='DDPG', eval_log_path=None, reset_num_timesteps=True)

## PPO with GAE from `stable_baseline3` 

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

total_t_steps = 20

model = PPO("MlpPolicy", env, verbose=1, gamma = 1.0, tensorboard_log=logdir, batch_size=2, n_steps=2)

# classstable_baselines3.ppo.PPO(policy, env, learning_rate=0.0003, n_steps=2048, 
#         batch_size=64, n_epochs=10, gamma=0.99, gae_lambda=0.95, clip_range=0.2, 
#         clip_range_vf=None, ent_coef=0.0, vf_coef=0.5, max_grad_norm=0.5, 
#         use_sde=False, sde_sample_freq=- 1, target_kl=None, tensorboard_log=None, 
#         create_eval_env=False, policy_kwargs=None, verbose=0, seed=None, device='auto', 
#         _init_setup_model=True)

for i in range(1,10):
    model.learn(total_timesteps=total_t_steps, reset_num_timesteps=False)

# learn(total_timesteps, callback=None, log_interval=1, eval_env=None, eval_freq=- 1, 
#       n_eval_episodes=5, tb_log_name='PPO', eval_log_path=None, reset_num_timesteps=True)

    model.save(f"{models_dir_ppo}/{total_t_steps*i}")

## From my `ddpg_agent.py` code

In [3]:
from ddpg_agent import DDPG_agent
agent = DDPG_agent(state_size, act_size, seed = 0)

In [4]:
## Save all rewards, energies and princip dims in files during training
def create_info_h5(agent, env):
    # Check if file exist and creat it
    i = 0
    while os.path.exists(f'run_{i}.hdf5'):
        i += 1
    dataFile = h5py.File(f'run_{i}.hdf5', 'a')
    
    # Create dataset to store info in hdf5 file
    info = {'alg':agent.name, 'env':env.unwrapped.spec.id}
    st = h5py.string_dtype(encoding='utf-8')
    dataFile.create_dataset('info', dtype=st)
    for k in info.keys():
        dataFile['info'].attrs[k] = info[k]

    # Create dataset to store hyperparams of the model in hdf5 file
    hyperparams = {'batch_size':agent.batch_size, 'bootstrap_size':agent.bootstrap_size \
                   , 'gamma':agent.gamma, 'tau':agent.tau,'lr_critic':agent.lr_critic \
                  , 'lr_actor':agent.lr_actor, 'update_every':agent.update_every \
                   , 'transfer_every':agent.transfer_every, 'num_update':agent.num_update \
                  , 'add_noise_every':agent.add_noise_every}
    dataFile.create_dataset('hyperparams', dtype='f')
    for k in hyperparams.keys():
        dataFile['hyperparams'].attrs[k] = hyperparams[k]
    
    # Create group for rewards, energies, princip dims, actor and critic model
    dataFile.create_group('sigmas')
    dataFile.create_group('rewards')
    dataFile.create_group('energies')
    dataFile.create_group('princip_dims')
    dataFile.create_group('actor_models')
    dataFile.create_group('critic_models')
    
    return dataFile

def save_all(dat_file, i_ep, sigmas_i_ep, rew_i_ep, en_i_ep, pri_dim_i_ep, act_model_i_ep, cr_model_i_ep):
    # Create datasets for rewards, energies, pri dim and store data in it 
    dat_file.create_dataset(f'sigmas_ep{i_ep}', dtype='f', data=sigmas_i_ep)
    dat_file.create_dataset(f'rew_ep_{i_ep}', dtype='f', data=rew_i_ep)
    dat_file.create_dataset(f'en_ep_{i_ep}', dtype='f', data=en_i_ep)
    dat_file.create_dataset(f'pri_dim_ep_{i_ep}', dtype='i', data=pri_dim_i_ep)
    
    # Store in actor models group the network params at each ep
    actor_model = torch.load(act_model_i_ep)
    dat_file['actor_models'].create_dataset(f'act_mod_{i_ep}', dtype='f')
    for k in actor_model.keys():
        dat_file['actor_models'][f'act_mod_{i_ep}'].attrs.create(name=k,data=actor_model[k].numpy())
    
    # Store in actor models group the network params at each ep
    critic_model = torch.load(cr_model_i_ep)
    dat_file['critic_models'].create_dataset(f'cri_mod_{i_ep}', dtype='f')
    for k in critic_model.keys():
        dat_file['critic_models'][f'cri_mod_{i_ep}'].attrs.create(name=k,data=critic_model[k].numpy())

def close_file(dat_file):
    dat_file.close()

In [5]:
## Run ddpg algs   
def run_ddpg(max_t_step = 10, n_episodes=10):
    
    # Create h5 file and store info about alg and its hypereparams
    dat_file = create_info_h5(agent, env)
    
    for i_ep in range(n_episodes):
        state = env.reset()
        agent.reset()
        rew_i_ep = []
        en_i_ep = []
        pri_dim_i_ep = []

        ## Training loop of each episode
        for t_step in range(max_t_step):
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state

            # Save rew, energies, princip dims, act and crit models
            rew_i_ep.append(reward)
            en_i_ep.append(state[0])
            pri_dim_i_ep.append(env.princp_dim)
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            if done:
                break
                
        ## Save data during training (to not lose the work done)
        save_all(dat_file=dat_file, i_ep=int(i_ep), sigmas_i_ep=env.actions_taken \
                 , rew_i_ep=rew_i_ep, en_i_ep=en_i_ep, pri_dim_i_ep=pri_dim_i_ep \
                 , act_model_i_ep='checkpoint_actor.pth', cr_model_i_ep='checkpoint_critic.pth')
        
        print('Episode {} ... Score: {:.3f}'.format(i_ep, np.sum(rew_i_ep)))

    close_file(dat_file)
    return dat_file

In [7]:
all_data = run_ddpg()

#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
#### CALL STEP #### 1
Action chosen at step:  [71.902405 43.021652 61.030563]
Basis size (it should be the same of full dim) =   1
With this action the energy is:  0.024218
With this action the full dim is:  1  and princip dim is:  1
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.1446431273843896
#### CALL STEP #### 2
Action chosen at step:  [75.91998  49.982918 67.659424]
Basis size (it should be the same of full dim) =   2
With this action the energy is:  0.0225171
With this action the full dim is:  2  and princip dim is:  2
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.0263634537435244
#### CALL STEP #### 3
Action chosen at step:  [54.047325 47.97421  43.36907 ]
Basis size (it should be the same of full dim) =   3
With this action the energy is:  0.019941
With this action the full dim is:  3  and princip dim is:  3
####

With this action the energy is:  0.0142177
With this action the full dim is:  5  and princip dim is:  5
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.449227629523989
#### CALL STEP #### 6
Action chosen at step:  [52.10719  52.719715 87.21936 ]
Basis size (it should be the same of full dim) =   6
With this action the energy is:  0.00902015
With this action the full dim is:  6  and princip dim is:  6
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.0877927904787281
#### CALL STEP #### 7
Action chosen at step:  [51.45754  55.8817   89.447174]
Basis size (it should be the same of full dim) =   7
With this action the energy is:  0.00813429
With this action the full dim is:  7  and princip dim is:  7
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.0261905612521414
#### CALL STEP #### 8
Action chosen at step:  [55.022636 44.805836 81.474304]
Basis size (it should be the same of full dim) =   8
With this action t

With this action the energy is:  -0.0248619
With this action the full dim is:  10  and princip dim is:  10
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  1.268346792261891
Episode 4 ... Score: -6.169
#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
#### CALL STEP #### 1
Action chosen at step:  [53.21033  38.890873 79.68896 ]
Basis size (it should be the same of full dim) =   1
With this action the energy is:  0.0244839
With this action the full dim is:  1  and princip dim is:  1
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.1631336707561974
#### CALL STEP #### 2
Action chosen at step:  [48.087555 34.47763  85.89944 ]
Basis size (it should be the same of full dim) =   2
With this action the energy is:  0.0239656
With this action the full dim is:  2  and princip dim is:  2
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.1270913667712126
#### CALL ST

With this action the energy is:  0.00872258
With this action the full dim is:  4  and princip dim is:  4
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.0670999322946422
#### CALL STEP #### 5
Action chosen at step:  [27.53262  34.254066 66.865654]
Basis size (it should be the same of full dim) =   5
With this action the energy is:  0.00315219
With this action the full dim is:  5  and princip dim is:  5
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.6797379996399613
#### CALL STEP #### 6
Action chosen at step:  [27.293844 38.447002 71.40506 ]
Basis size (it should be the same of full dim) =   6
With this action the energy is:  -0.00396021
With this action the full dim is:  6  and princip dim is:  6
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.1851455187161566
#### CALL STEP #### 7
Action chosen at step:  [20.830357 38.97498  71.64119 ]
Basis size (it should be the same of full dim) =   7
With this actio

With this action the energy is:  -0.0979385
With this action the full dim is:  9  and princip dim is:  9
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  6.350054358129453
#### CALL STEP #### 10
Action chosen at step:  [21.708878 54.274143 82.9246  ]
Basis size (it should be the same of full dim) =   10
With this action the energy is:  -0.098064
With this action the full dim is:  10  and princip dim is:  10
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  6.358781560811521
Episode 9 ... Score: 54.015


In [ ]:
import matplotlib.pyplot as plt

scores = np.loadtxt('scores_RL.out')
energies = np.loadtxt('energies_RL.out')
dim = np.loadtxt('princip_dim_RL.out')

plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')

In [ ]:
plt.plot(np.arange(len(energies)), energies)
plt.ylabel('Eenergies (mK)')
plt.xlabel('Episode #')

In [ ]:
plt.plot(np.arange(len(dim)), dim)
plt.ylabel('dim (mK)')
plt.xlabel('Episode #')

In [ ]:
rewards = []

for i in range(10):
    rew_i_episode = []
    for j in range(20):
        rew_i_episode.append(j)
    rewards.append(rew_i_episode)
    print('Episode {} ... Score: {:.3f}'.format(i, np.sum(rewards[i])))
    
name_rewards = 'rewards_RL.out'
file_rewards = open(name_rewards,'w')
np.savetxt(file_rewards, rewards, fmt="%f")
file_rewards.close()

## Random search as in original SVM

In [ ]:
state = env.reset()
scores = []
step = 0
score = 0.0

while True:
    print(".....STEP.....", step)
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)
    step = step + 1
    score += reward
    scores.append(score)
    state = next_state
    if done:
        break